In [20]:
from scipy.sparse import random
import numpy as np
from scipy.sparse.linalg import svds

user_item_matrix = random(10000, 1000, density=0.01, format='coo', dtype=np.int32, random_state=42)
user_item_matrix = user_item_matrix.asfptype()

U, s, V = svds(user_item_matrix, k=40)

print(f'Размерности матриц U={U.shape}, s={s.shape}, V={V.shape}')
print(f'Типы матриц U={type(U)}, s={type(s)}, V={type(V)}')

Размерности матриц U=(10000, 40), s=(40,), V=(40, 1000)
Типы матриц U=<class 'numpy.ndarray'>, s=<class 'numpy.ndarray'>, V=<class 'numpy.ndarray'>


In [24]:
import h5py

with h5py.File('s_matrix.hdf5') as f:
    h5py_dset = f.create_dataset("default", data=s)
    print(f'Создали датасет в менеджере контакста {h5py_dset}')
print(f'Тут датаст не доступен {h5py_dset}')

Создали датасет в менеджере контакста <HDF5 dataset "default": shape (40,), type "<f8">
Тут датаст не доступен <Closed HDF5 dataset>


In [31]:
with h5py.File('complex_dataset.hdf5','w') as f:
    raw = f.create_group('source_data')
    raw.create_dataset('complex_dataset.hdf5', data=np.random.random(1000))

    processed = raw.create_group('model_data')

    processed.create_dataset('user_factors', data=U, dtype=np.float32, compression='gzip')
    processed.create_dataset('eigen_values', data=s, dtype=np.float32, compression='gzip')
    processed.create_dataset('item_factors', data=V, dtype=np.float32, compression='gzip')

    print(f'Уровень группы:\t\t{raw.items}')
    print(f'Уровень подгруппы\t{processed.items}')
    print(f'Имена элементов в подгруппе:\n')
    group_names = [i.name for i in f['source_data/model_data'].values()]
    for name in group_names:
        print(name)

Уровень группы:		<bound method MappingHDF5.items of <HDF5 group "/source_data" (2 members)>>
Уровень подгруппы	<bound method MappingHDF5.items of <HDF5 group "/source_data/model_data" (3 members)>>
Имена элементов в подгруппе:

/source_data/model_data/eigen_values
/source_data/model_data/item_factors
/source_data/model_data/user_factors


In [32]:
with h5py.File('s_matrix.hdf5', 'r') as f:
    print("Доступные ключи %s\n" % list(f.keys()))
    data = f['default']
    print("min={}, \nmax={}, \nslice={}\n".format(min(data), max(data), data[10:15]))
    print("Попытка воспользоваться функциями numpy...")
    try:
        print(data.min())
    except AttributeError:
        print("Не вышло =(")
    data_copy = data[:]
print("Типизация датасетов: data={}, data_copy={}\n".format(data, type(data_copy)))

# Чтение иерархических датасетов
with h5py.File('complex_dataset.hdf5', 'r') as f:
    U_hdf = f['source_data/model_data/user_factors'][:]
    s_hdf = f['source_data/model_data/eigen_values'][:]
    V_hdf = f['source_data/model_data/item_factors'][:]
    
print("Размерности матриц U={}, s={}, V={}".format(U_hdf.shape, s_hdf.shape, V_hdf.shape))
print("Типизация матриц U={}, s={}, V={}".format(type(U_hdf), type(s_hdf), type(V_hdf)))

Доступные ключи ['default']

min=15967970843.644152, 
max=16869307715.289255, 
slice=[1.61010982e+10 1.61161420e+10 1.61308403e+10 1.61512488e+10
 1.61722312e+10]

Попытка воспользоваться функциями numpy...
Не вышло =(
Типизация датасетов: data=<Closed HDF5 dataset>, data_copy=<class 'numpy.ndarray'>

Размерности матриц U=(10000, 40), s=(40,), V=(40, 1000)
Типизация матриц U=<class 'numpy.ndarray'>, s=<class 'numpy.ndarray'>, V=<class 'numpy.ndarray'>
